In [1]:
import gc, random, sqlite3, shutil, pathlib as pl, sys, math
from typing import Sequence, Dict

import numpy as np
import pandas as pd
import pyarrow as pa, pyarrow.parquet as pq
import torch, torch.nn as nn, torch.optim as optim
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

pd.set_option("display.max_rows", 50)
pd.set_option("display.float_format", "{:0.4f}".format)

In [2]:
from pathlib import Path
from google.colab import drive, files

drive.mount("/content/drive", force_remount=False)

CSV_NAME    = "datashare.csv"
SQLITE_NAME = "tidy_finance.sqlite"

SEARCH_ROOTS = [
    Path("/content"),
    Path("/content/drive/MyDrive"),
]

CSV_PATH   = None
SQLITE_DB  = None

def find_one(root: Path, fname: str) -> Path | None:
    """Return the first match for fname found below root, else None."""
    try:
        return next(root.rglob(fname))
    except StopIteration:
        return None

for root in SEARCH_ROOTS:
    CSV_PATH   = CSV_PATH   or find_one(root, CSV_NAME)
    SQLITE_DB  = SQLITE_DB  or find_one(root, SQLITE_NAME)
    if CSV_PATH and SQLITE_DB:
        break

assert CSV_PATH and SQLITE_DB, "Files missing!"
print(f"✓ CSV  ➜ {CSV_PATH}")
print(f"✓ SQL  ➜ {SQLITE_DB}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ CSV  ➜ /content/drive/MyDrive/RAShip/datashare.csv
✓ SQL  ➜ /content/drive/MyDrive/RAShip/tidy_finance.sqlite


In [3]:
SEED_BASE = 42
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"

DATA_DIR   = pl.Path("/content/data")
CHAR_DIR   = DATA_DIR / "char"
MERGED_DIR = DATA_DIR / "merged"
FINAL_DIR  = DATA_DIR / "final"
for d in (DATA_DIR, CHAR_DIR, MERGED_DIR, FINAL_DIR):
    d.mkdir(exist_ok=True)

ARCH: Dict[str, Sequence[int]] = {
    "nn1": [32],
    "nn2": [32, 16],
    "nn3": [32, 16, 8],
    "nn4": [32, 16, 8, 4],
    "nn5": [32, 16, 8, 4, 2],
}

torch.backends.cudnn.benchmark = True
pd.options.mode.chained_assignment = None

In [4]:
def rank_to_unit(x: pd.Series) -> pd.Series:
    r = x.rank(method="first", na_option="keep")
    return 2 * ((r - 1) / (r.max() - 1) - 0.5)


def prep_chars(csv_path: pl.Path, out_dir: pl.Path):
    if list(out_dir.glob("*.parquet")):
        print("↳ Characteristic shards already exist – skipping step 1")
        return

    print("↳ Building characteristic shards …")
    df = pd.read_csv(csv_path)
    df["month"] = pd.to_datetime(df["DATE"], format="%Y%m%d")\
                     .dt.to_period("M").dt.to_timestamp()
    df.drop(columns="DATE", inplace=True)

    id_cols   = ["permno", "month"] + (["sic2"] if "sic2" in df.columns else [])
    char_cols = [c for c in df.columns if c not in id_cols]
    df.rename(columns={c: f"characteristic_{c}" for c in char_cols}, inplace=True)

    rank_cols = [c for c in df.columns if c.startswith("characteristic_")]
    df[rank_cols] = (df.groupby("month", group_keys=False)[rank_cols]
                       .apply(lambda g: g.apply(rank_to_unit)))

    med = df.groupby("month")[rank_cols].transform("median")
    df[rank_cols] = df[rank_cols].fillna(med).fillna(0)

    df["year"] = df["month"].dt.year
    for yr, grp in tqdm(df.groupby("year"), desc="write char", unit="yr"):
        pq.write_table(pa.Table.from_pandas(grp.drop(columns="year")),
                       out_dir / f"{yr}.parquet")

prep_chars(CSV_PATH, CHAR_DIR)


↳ Characteristic shards already exist – skipping step 1


In [5]:
def load_macro_from_google() -> pd.DataFrame:
    sheet_id = "1bM7vCWd3WOt95Sf9qjLPZjoiafgF_8EG"
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=macro_predictors.xlsx"
    raw = pd.read_csv(url, thousands=",")
    macro = (raw.assign(
                 month=lambda x: pd.to_datetime(x["yyyymm"], format="%Y%m"),
                 dp  = np.log(raw["D12"]) - np.log(raw["Index"]),
                 ep  = np.log(raw["E12"]) - np.log(raw["Index"]),
                 bm  = raw["b/m"],
                 ntis= raw["ntis"],
                 tbl = raw["tbl"],
                 tms = raw["lty"] - raw["tbl"],
                 dfy = raw["BAA"] - raw["AAA"],
                 svar= raw["svar"])
             .get(["month","dp","ep","bm","ntis","tbl","tms","dfy","svar"])
             .dropna())
    macro["month"] = macro["month"].dt.to_period("M").dt.to_timestamp()
    macro = macro.rename(columns=lambda c: f"macro_{c}" if c!="month" else c)
    lag = macro.copy(); lag["month"] += pd.offsets.MonthEnd(1)
    return macro[["month"]].merge(lag, on="month")

In [6]:
def sql_df(q: str) -> pd.DataFrame:
    with sqlite3.connect(SQLITE_DB) as con:
        return pd.read_sql_query(q, con, parse_dates=["month"])

def build_merged_shards():
    if list(MERGED_DIR.glob("*.parquet")):
        print("↳ Merged shards already exist – skipping step 3")
        return

    print("↳ Merging CRSP + macro + characteristics …")
    crsp  = sql_df("SELECT month, permno, mktcap_lag, ret_excess FROM crsp_monthly")

    with sqlite3.connect(SQLITE_DB) as con:
        has_macro = "macro_predictors" in [t for (t,) in
                                           con.execute("SELECT name FROM sqlite_master "
                                                       "WHERE type='table'")]
    macro = sql_df("SELECT * FROM macro_predictors") if has_macro else load_macro_from_google()
    macro["month"] = macro["month"].dt.to_period("M").dt.to_timestamp()
    macro = macro.rename(columns=lambda c: f"macro_{c}" if c!="month" else c)
    lag = macro.copy(); lag["month"] += pd.offsets.MonthEnd(1)
    macro = macro[["month"]].merge(lag, on="month")

    for p in tqdm(sorted(CHAR_DIR.glob("*.parquet")), desc="merge", unit="yr"):
        feats = pq.read_table(p).to_pandas()
        merged = (feats
                  .merge(crsp,  on=["month","permno"], how="left")
                  .merge(macro, on="month",          how="left")
                  .assign(macro_intercept=1.0))
        pq.write_table(pa.Table.from_pandas(merged), MERGED_DIR / p.name)
        del feats, merged; gc.collect()

build_merged_shards()


↳ Merged shards already exist – skipping step 3


In [7]:
# characteristic × macro interactions
def build_interactions():
    if list(FINAL_DIR.glob("*.parquet")):
        print("↳ Interaction shards already exist – skipping step 4")
        return

    print("↳ Creating characteristic × macro interactions (vectorised) …")
    for p in tqdm(sorted(MERGED_DIR.glob("*.parquet")), desc="interactions", unit="yr"):
        df = pq.read_table(p).to_pandas()

        char_cols  = [c for c in df.columns if c.startswith("characteristic_")]
        macro_cols = [c for c in df.columns if c.startswith("macro_")]

        # ------- vectorised interactions -------------------------------------
        #  X_char :  n × C   ;  X_macro : n × M
        X_char  = df[char_cols].to_numpy(dtype="float32")
        X_macro = df[macro_cols].to_numpy(dtype="float32")
        #  broadcast, multiply, reshape to n × (C·M)
        inter   = (X_char[:, :, None] * X_macro[:, None, :]).reshape(len(df), -1)

        inter_cols = [f"{c}__x__{m}" for c in char_cols for m in macro_cols]
        df_inter   = pd.DataFrame(inter, columns=inter_cols, index=df.index)

        # join once → no fragmentation warning
        df = pd.concat([df, df_inter], axis=1, copy=False)

        # one‑hot SIC‑2
        if "sic2" in df.columns:
            df = pd.get_dummies(df, columns=["sic2"], prefix="sic2", dtype="int8")

        pq.write_table(pa.Table.from_pandas(df), FINAL_DIR / p.name)
        del df, df_inter, inter, X_char, X_macro
        gc.collect()

build_interactions()


↳ Interaction shards already exist – skipping step 4


In [8]:
def masks(df: pd.DataFrame, test_year: int):
    y = df["month"].dt.year
    return ((y>=1957)&(y<=test_year-13),
            (y>=test_year-12)&(y<=test_year-1),
            y==test_year)


In [9]:
class FFN(nn.Module):
    def __init__(self, d_in:int, widths:Sequence[int]):
        super().__init__()
        seq, prev = [], d_in
        for w in widths:
            seq += [nn.Linear(prev,w), nn.ReLU(), nn.BatchNorm1d(w)]; prev=w
        seq.append(nn.Linear(prev,1)); self.net = nn.Sequential(*seq)
    def forward(self,x): return self.net(x).squeeze(-1)

def fit_nn(train, val, arch, seed):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    cols_drop=["permno","month","ret_excess","mktcap_lag"]

    Xtr,ytr = train.drop(cols_drop,axis=1).values.astype("float32"), train["ret_excess"].values.astype("float32")
    Xva,yva = val.drop(cols_drop,axis=1).values.astype("float32"), val["ret_excess"].values.astype("float32")

    sc = StandardScaler().fit(Xtr); Xtr,Xva = sc.transform(Xtr),sc.transform(Xva)

    ds_tr = torch.utils.data.TensorDataset(torch.as_tensor(Xtr), torch.as_tensor(ytr))
    ds_va = torch.utils.data.TensorDataset(torch.as_tensor(Xva), torch.as_tensor(yva))
    dl_tr = torch.utils.data.DataLoader(ds_tr,batch_size=8192,shuffle=True,pin_memory=True)
    dl_va = torch.utils.data.DataLoader(ds_va,batch_size=8192,pin_memory=True)

    net = FFN(Xtr.shape[1],ARCH[arch]).to(DEVICE)
    opt,crit = optim.Adam(net.parameters(),lr=1e-3), nn.MSELoss()
    best=float("inf"); waited=0
    for _ in range(100):
        net.train()
        for xb,yb in dl_tr:
            xb,yb=xb.to(DEVICE),yb.to(DEVICE)
            opt.zero_grad(); crit(net(xb),yb).backward(); opt.step()
        val_loss=0; net.eval()
        with torch.no_grad():
            for xb,yb in dl_va:
                val_loss+=crit(net(xb.to(DEVICE)), yb.to(DEVICE)).item()*len(yb)
        val_loss/=len(ds_va)
        if val_loss<best: best,best_state,waited=val_loss,net.state_dict(),0
        else:
            waited+=1
            if waited>=5: break
    net.load_state_dict(best_state)
    return net,sc

def predict(net,sc,df):
    X=sc.transform(df.drop(["permno","month","ret_excess","mktcap_lag"],axis=1).values.astype("float32"))
    with torch.no_grad():
        pr=net(torch.as_tensor(X).to(DEVICE)).cpu().numpy()
    out=df[["permno","month","mktcap_lag","ret_excess"]].copy(); out["pred"]=pr; return out


In [ ]:
#  Cell 10 – OOS training loop
import time

N_ENSEMBLE   = 5         # set to 2‑3 while prototyping, 10 for full replication
MAX_EPOCHS   = 20
PATIENCE     = 5
MIN_DELTA    = 1e-5
PRED         = []

def fit_nn(train, val, arch, seed, echo=False):
    """Feed‑forward net with early‑stop; echo=True prints per‑epoch val‑MSE."""
    cols_drop = ["permno", "month", "ret_excess", "mktcap_lag"]

    Xtr = train.drop(columns=cols_drop).to_numpy("float32")
    ytr = train["ret_excess"].to_numpy("float32")
    Xva = val.drop(columns=cols_drop).to_numpy("float32")
    yva = val["ret_excess"].to_numpy("float32")

    sc = StandardScaler().fit(Xtr)
    Xtr, Xva = sc.transform(Xtr), sc.transform(Xva)

    ds_tr = torch.utils.data.TensorDataset(torch.as_tensor(Xtr), torch.as_tensor(ytr))
    ds_va = torch.utils.data.TensorDataset(torch.as_tensor(Xva), torch.as_tensor(yva))
    dl_tr = torch.utils.data.DataLoader(ds_tr, batch_size=8192, shuffle=True, pin_memory=True)
    dl_va = torch.utils.data.DataLoader(ds_va, batch_size=8192, pin_memory=True)

    net = FFN(Xtr.shape[1], ARCH[arch]).to(DEVICE)
    opt, crit = optim.Adam(net.parameters(), lr=1e-3), nn.MSELoss()

    best, waited = float("inf"), 0
    for epoch in range(MAX_EPOCHS):
        net.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad(); crit(net(xb), yb).backward(); opt.step()

        # validation
        net.eval(); val_loss = 0.0
        with torch.no_grad():
            for xb, yb in dl_va:
                val_loss += crit(net(xb.to(DEVICE)), yb.to(DEVICE)).item() * len(yb)
        val_loss /= len(ds_va)

        if echo:
            print(f"    epoch {epoch+1:02d}  val‑MSE {val_loss:,.4e}")

        if val_loss < best - MIN_DELTA:
            best, best_state, waited = val_loss, net.state_dict(), 0
        else:
            waited += 1
            if waited >= PATIENCE:
                break

    net.load_state_dict(best_state)
    return net, sc

print(f"GPU detected: {torch.cuda.get_device_name(0) if DEVICE=='cuda' else 'CPU‑only'}")

for yr in tqdm(range(2000, 2022), desc="OOS", unit="yr"):
    t0 = time.time()

    shards = [pq.read_table(p).to_pandas()
              for p in FINAL_DIR.glob("*.parquet") if int(p.stem) <= yr]
    df_all = pd.concat(shards, ignore_index=True); del shards

    sic_cols = [c for c in df_all.columns if c.startswith("sic2_")]
    if sic_cols:
        df_all[sic_cols] = df_all[sic_cols].fillna(0).astype("int8")
    df_all = df_all.fillna(0)

    tr, va, te = masks(df_all, yr)
    if df_all[tr].empty or df_all[va].empty:
        print(f"{yr}: skipped – no data")
        continue

    preds = []
    for s in range(N_ENSEMBLE):
        echo = (yr == 2000 and s == 0)   # only the first model is verbose
        net, sc = fit_nn(df_all[tr], df_all[va], "nn5",
                         SEED_BASE + s, echo=echo)
        preds.append(predict(net, sc, df_all[te]))

    tmp = preds[0].copy()
    tmp["pred"] = np.mean([p_["pred"].values for p_ in preds], axis=0)

    if np.allclose(tmp["pred"].values, 0.0):
        raise RuntimeError(f"Year {yr}: predictions collapsed to zeros.")

    PRED.append(tmp)
    print(f"✓ {yr}  {N_ENSEMBLE} nets  {time.time() - t0:5.1f}s  "
          f"val‑MSE best {min(p['pred'].var() for p in preds):.2e}")

    del df_all, tmp, preds
    gc.collect()

pred_df = pd.concat(PRED, ignore_index=True)


GPU detected: Tesla T4


OOS:   0%|          | 0/22 [00:00<?, ?yr/s]

    epoch 01  val‑MSE 9.0402e-03
    epoch 02  val‑MSE 6.1669e-04
    epoch 03  val‑MSE 2.3611e-04
    epoch 04  val‑MSE 1.7142e-04
    epoch 05  val‑MSE 1.0143e-04
    epoch 06  val‑MSE 8.4170e-05
    epoch 07  val‑MSE 6.1710e-05
    epoch 08  val‑MSE 5.4876e-05
    epoch 09  val‑MSE 4.3366e-05
    epoch 10  val‑MSE 3.9045e-05
    epoch 11  val‑MSE 3.5081e-05
    epoch 12  val‑MSE 3.2962e-05
    epoch 13  val‑MSE 3.0357e-05
    epoch 14  val‑MSE 2.7531e-05
    epoch 15  val‑MSE 2.5595e-05
    epoch 16  val‑MSE 2.3199e-05
    epoch 17  val‑MSE 2.4425e-05


OOS:   5%|▍         | 1/22 [1:00:53<21:18:41, 3653.43s/yr]

✓ 2000  5 nets  3653.2s  val‑MSE best 2.73e-07


OOS:   9%|▉         | 2/22 [2:11:18<22:09:48, 3989.40s/yr]

✓ 2001  5 nets  4224.4s  val‑MSE best 4.26e-06


OOS:  14%|█▎        | 3/22 [3:03:20<18:57:58, 3593.60s/yr]

✓ 2002  5 nets  3122.4s  val‑MSE best 6.07e-16


In [1]:
def assign_decile(series): return pd.qcut(series,10,labels=False,duplicates="drop")+1
pred_df["portfolio"]=pred_df.groupby("month",group_keys=False)["pred"].apply(assign_decile)

panel=(pred_df.groupby(["portfolio","month"])[["ret_excess","pred"]]
         .mean().reset_index())
hi,lo = panel[panel["portfolio"]==10], panel[panel["portfolio"]==1]
hml = hi.set_index("month") - lo.set_index("month"); hml.reset_index(inplace=True); hml["portfolio"]="H‑L"

panel  = pd.concat([panel,hml],ignore_index=True)
summary=(panel.groupby("portfolio")
              .agg(predicted_mean=("pred","mean"),
                   realized_mean=("ret_excess","mean"),
                   realized_sd=("ret_excess","std")))
summary["sharpe"]=summary["realized_mean"]/summary["realized_sd"]

print("\n================  Out‑of‑sample performance 2000‑2021  ================\n")
print(summary)

NameError: name 'pred_df' is not defined